In [1]:
import pyspark
import os
import zipfile

In [2]:
from io import StringIO
import csv
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [3]:
from pyspark import *
import numpy as np

In [2]:
# unzip file from local/virtual machine
# zip_ref = zipfile.ZipFile("/home/zxr_nju/data/events.csv.zip", 'r')
# zip_ref.extractall("/home/zxr_nju/data/")
# zip_ref.close()

### Encode Data Using Spark

In [4]:
sc = pyspark.SparkContext.getOrCreate()

In [5]:
ss = SparkSession.builder.getOrCreate()

In [57]:
rdd = sc.textFile("/home/zxr_nju/data/events.csv")

In [58]:
rdd_events = rdd.map(lambda x: x.split(","))

In [59]:
rdd_events.take(3)

[['app_id',
  'session_id',
  'event',
  'event_timestamp',
  'event_value',
  'user_id_hash'],
 ['4724682771660800',
  '5558845121177764917',
  '45',
  '1542215397132',
  '0.0',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006'],
 ['4724682771660800',
  '5558845121177764917',
  '45',
  '1542215484895',
  '0.0',
  '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']]

In [60]:
header = rdd_events.first()
header

['app_id',
 'session_id',
 'event',
 'event_timestamp',
 'event_value',
 'user_id_hash']

In [61]:
lines = rdd_events.filter(lambda x: x!=header)

In [62]:
lines.first()

['4724682771660800',
 '5558845121177764917',
 '45',
 '1542215397132',
 '0.0',
 '9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']

In [63]:
user_id_map = np.load("/home/zxr_nju/user_id_map.npy").item()

In [64]:
def user_map(s):
    return str(user_id_map[s])

In [14]:
# def timestamp_convert(s):
#     return str(int(s)/(3.6*(10**6)))

In [75]:
# def string_convert(s):
#     return int(s)

In [65]:
lines = lines.map(lambda x: [x[0], x[1], x[2], x[3], x[4], user_map(x[5])])

In [49]:
lines.take(5)

[['4724682771660800',
  '5558845121177764917',
  '45',
  '1542215397132',
  '0.0',
  '0'],
 ['4724682771660800',
  '5558845121177764917',
  '45',
  '1542215484895',
  '0.0',
  '0'],
 ['4724682771660800',
  '7689508378645584666',
  '.m5100869650219008',
  '1541124410372',
  '0.0',
  '0'],
 ['4724682771660800', '2201961907282901522', '4', '1543713091129', '0.0', '0'],
 ['4724682771660800', '2201961907282901522', '6', '1543713093116', '0.0', '0']]

In [66]:
# fields = [StructField(field_name, StringType(), True) for field_name in header]
# schema = StructType(fields)

In [67]:
schema = StructType([
        StructField("app_id", StringType(), True),
        StructField("session_id", StringType(), True),
        StructField("event", StringType(), True),
        StructField("event_timestamp", StringType(), True),
        StructField("event_value", StringType(), True),
        StructField("user_id_hash", StringType(), True),
    ])

In [68]:
events_df = ss.createDataFrame(lines, schema=schema)

In [69]:
events_df.show()

+----------------+-------------------+------------------+---------------+-----------+------------+
|          app_id|         session_id|             event|event_timestamp|event_value|user_id_hash|
+----------------+-------------------+------------------+---------------+-----------+------------+
|4724682771660800|5558845121177764917|                45|  1542215397132|        0.0|           0|
|4724682771660800|5558845121177764917|                45|  1542215484895|        0.0|           0|
|4724682771660800|7689508378645584666|.m5100869650219008|  1541124410372|        0.0|           0|
|4724682771660800|2201961907282901522|                 4|  1543713091129|        0.0|           0|
|4724682771660800|2201961907282901522|                 6|  1543713093116|        0.0|           0|
|4724682771660800|2201961907282901522|                 5|  1543713101700|        1.0|           0|
|4724682771660800|2201961907282901522|                45|  1543713186756|        0.0|           0|
|472468277

In [77]:
from pyspark.sql.functions import *

In [79]:
events_new = events_df.select("session_id", "event", from_unixtime((events_df.event_timestamp.cast('bigint')/1000))\
                              .cast('timestamp').alias('timestamp'), "event_value", "user_id_hash")

In [80]:
events_new.show()

+-------------------+------------------+-------------------+-----------+------------+
|         session_id|             event|          timestamp|event_value|user_id_hash|
+-------------------+------------------+-------------------+-----------+------------+
|5558845121177764917|                45|2018-11-14 17:09:57|        0.0|           0|
|5558845121177764917|                45|2018-11-14 17:11:24|        0.0|           0|
|7689508378645584666|.m5100869650219008|2018-11-02 02:06:50|        0.0|           0|
|2201961907282901522|                 4|2018-12-02 01:11:31|        0.0|           0|
|2201961907282901522|                 6|2018-12-02 01:11:33|        0.0|           0|
|2201961907282901522|                 5|2018-12-02 01:11:41|        1.0|           0|
|2201961907282901522|                45|2018-12-02 01:13:06|        0.0|           0|
|2201961907282901522|                45|2018-12-02 01:13:26|        0.0|           0|
|2201961907282901522|                45|2018-12-02 01:

In [82]:
from pyspark.sql.functions import col, hour
split_col = pyspark.sql.functions.split(events_new["timestamp"], " ")
events_new = events_new.withColumn("date", split_col.getItem(0))
events_new = events_new.withColumn("time", split_col.getItem(1))

In [83]:
events_new.show()

+-------------------+------------------+-------------------+-----------+------------+----------+--------+
|         session_id|             event|          timestamp|event_value|user_id_hash|      date|    time|
+-------------------+------------------+-------------------+-----------+------------+----------+--------+
|5558845121177764917|                45|2018-11-14 17:09:57|        0.0|           0|2018-11-14|17:09:57|
|5558845121177764917|                45|2018-11-14 17:11:24|        0.0|           0|2018-11-14|17:11:24|
|7689508378645584666|.m5100869650219008|2018-11-02 02:06:50|        0.0|           0|2018-11-02|02:06:50|
|2201961907282901522|                 4|2018-12-02 01:11:31|        0.0|           0|2018-12-02|01:11:31|
|2201961907282901522|                 6|2018-12-02 01:11:33|        0.0|           0|2018-12-02|01:11:33|
|2201961907282901522|                 5|2018-12-02 01:11:41|        1.0|           0|2018-12-02|01:11:41|
|2201961907282901522|                45|2018-1

In [84]:
events_new = events_new.withColumn("date_tmstp", (col("timestamp").cast("timestamp")))
events_new = events_new.withColumn("hour", hour("date_tmstp"))

In [85]:
events_new.show()

+-------------------+------------------+-------------------+-----------+------------+----------+--------+-------------------+----+
|         session_id|             event|          timestamp|event_value|user_id_hash|      date|    time|         date_tmstp|hour|
+-------------------+------------------+-------------------+-----------+------------+----------+--------+-------------------+----+
|5558845121177764917|                45|2018-11-14 17:09:57|        0.0|           0|2018-11-14|17:09:57|2018-11-14 17:09:57|  17|
|5558845121177764917|                45|2018-11-14 17:11:24|        0.0|           0|2018-11-14|17:11:24|2018-11-14 17:11:24|  17|
|7689508378645584666|.m5100869650219008|2018-11-02 02:06:50|        0.0|           0|2018-11-02|02:06:50|2018-11-02 02:06:50|   2|
|2201961907282901522|                 4|2018-12-02 01:11:31|        0.0|           0|2018-12-02|01:11:31|2018-12-02 01:11:31|   1|
|2201961907282901522|                 6|2018-12-02 01:11:33|        0.0|           

In [86]:
events_new = events_new.select("session_id", "event", "event_value", "user_id_hash", "date", "hour")

In [87]:
events_new.show()

+-------------------+------------------+-----------+------------+----------+----+
|         session_id|             event|event_value|user_id_hash|      date|hour|
+-------------------+------------------+-----------+------------+----------+----+
|5558845121177764917|                45|        0.0|           0|2018-11-14|  17|
|5558845121177764917|                45|        0.0|           0|2018-11-14|  17|
|7689508378645584666|.m5100869650219008|        0.0|           0|2018-11-02|   2|
|2201961907282901522|                 4|        0.0|           0|2018-12-02|   1|
|2201961907282901522|                 6|        0.0|           0|2018-12-02|   1|
|2201961907282901522|                 5|        1.0|           0|2018-12-02|   1|
|2201961907282901522|                45|        0.0|           0|2018-12-02|   1|
|2201961907282901522|                45|        0.0|           0|2018-12-02|   1|
|2201961907282901522|                45|        0.0|           0|2018-12-02|   1|
|220196190728290

In [88]:
from pyspark.sql.functions import date_format
events_new = events_new.select("session_id", "event", "event_value", "user_id_hash", "date", "hour",\
                               date_format('date', 'u').alias('dow_number'), \
                               date_format('date', 'E').alias('dow_string'))

In [89]:
events_new.show()

+-------------------+------------------+-----------+------------+----------+----+----------+----------+
|         session_id|             event|event_value|user_id_hash|      date|hour|dow_number|dow_string|
+-------------------+------------------+-----------+------------+----------+----+----------+----------+
|5558845121177764917|                45|        0.0|           0|2018-11-14|  17|         3|       Wed|
|5558845121177764917|                45|        0.0|           0|2018-11-14|  17|         3|       Wed|
|7689508378645584666|.m5100869650219008|        0.0|           0|2018-11-02|   2|         5|       Fri|
|2201961907282901522|                 4|        0.0|           0|2018-12-02|   1|         7|       Sun|
|2201961907282901522|                 6|        0.0|           0|2018-12-02|   1|         7|       Sun|
|2201961907282901522|                 5|        1.0|           0|2018-12-02|   1|         7|       Sun|
|2201961907282901522|                45|        0.0|           0

In [90]:
events_new.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- event: string (nullable = true)
 |-- event_value: string (nullable = true)
 |-- user_id_hash: string (nullable = true)
 |-- date: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- dow_number: string (nullable = true)
 |-- dow_string: string (nullable = true)



In [113]:
sc.stop()

/home/zxr_nju/anaconda3/lib/python3.5/site-packages/pyspark/context.py:428: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning


### Encoding Data Using Pandas

In [6]:
events_new.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 5 columns):
session_id         int64
event              object
event_timestamp    float64
event_value        float64
user_id_hash       int64
dtypes: float64(2), int64(2), object(1)
memory usage: 9.7 GB


In [7]:
events = pd.read_csv("~/data/events.csv")

MemoryError: 

In [6]:
events.to_feather("/tmp/events")

In [6]:
events.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [7]:
events.describe()

,app_id,session_id,event_timestamp,event_value
count,1.119466e+08,1.119466e+08,1.119466e+08,1.119465e+08
mean,4.724683e+15,4.615011e+18,1.541643e+12,1.476199e+04
std,1.056559e+07,2.662939e+18,1.639665e+09,7.948289e+06
min,4.724683e+15,2.335879e+12,1.538377e+12,0.000000e+00
25%,4.724683e+15,2.310347e+18,1.540306e+12,0.000000e+00
50%,4.724683e+15,4.617399e+18,1.541696e+12,0.000000e+00
75%,4.724683e+15,6.921542e+18,1.542983e+12,0.000000e+00
max,4.724683e+15,9.223371e+18,1.544832e+12,4.294967e+09


In [9]:
events.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 6 columns):
app_id             int64
session_id         int64
event              object
event_timestamp    int64
event_value        float64
user_id_hash       object
dtypes: float64(1), int64(3), object(2)
memory usage: 22.3 GB


In [24]:
import numpy as np

In [8]:
events["event_timestamp"] = events["event_timestamp"]/(3.6*(10**6))

In [9]:
events.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,428393.165870,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,5558845121177764917,45,428393.190249,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,7689508378645584666,.m5100869650219008,428090.113992,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,2201961907282901522,4,428809.191980,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,2201961907282901522,6,428809.192532,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [13]:
for i in range(len(events)):
    hashing_value = events["user_id_hash"].values[i]
    events["user_id_hash"].values[i] = user_id_map[hashing_value]

In [14]:
events.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,428393.165870,0.0,0
1,4724682771660800,5558845121177764917,45,428393.190249,0.0,0
2,4724682771660800,7689508378645584666,.m5100869650219008,428090.113992,0.0,0
3,4724682771660800,2201961907282901522,4,428809.191980,0.0,0
4,4724682771660800,2201961907282901522,6,428809.192532,0.0,0


In [15]:
events.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 6 columns):
app_id             int64
session_id         int64
event              object
event_timestamp    float64
event_value        float64
user_id_hash       object
dtypes: float64(2), int64(2), object(2)
memory usage: 13.8 GB


In [20]:
events.to_feather("/tmp/events")

In [4]:
events_df = pd.read_feather("/tmp/events").drop(['app_id'], axis=1)

In [5]:
events_df.head()

,session_id,event,event_timestamp,event_value,user_id_hash
0,5558845121177764917,45,428393.165870,0.0,0
1,5558845121177764917,45,428393.190249,0.0,0
2,7689508378645584666,.m5100869650219008,428090.113992,0.0,0
3,2201961907282901522,4,428809.191980,0.0,0
4,2201961907282901522,6,428809.192532,0.0,0


In [6]:
events_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 5 columns):
session_id         int64
event              object
event_timestamp    float64
event_value        float64
user_id_hash       int64
dtypes: float64(2), int64(2), object(1)
memory usage: 9.7 GB


In [7]:
events_df.session_id.nunique()

5009421

In [7]:
events_df.event_value.nunique()

88

In [8]:
events_df.event.nunique()

109

In [10]:
unique_sessions = list(events_df.session_id.unique())

In [11]:
session_mapping = list(np.arange(0, 5009421, 1))

In [12]:
session_id_map = {}

In [9]:
for u,m in zip(unique_sessions, session_mapping):
    session_id_map[u] = m

NameError: name 'unique_sessions' is not defined

In [30]:
session_id_map

{7058144086722514667: 3638706,
 6200027727731359744: 611760,
 5238444851531350017: 1873931,
 3538545855248203779: 3109648,
 974022461290971141: 1263240,
 7652611076072144900: 987543,
 8171190004765163519: 4780756,
 6625980323844849676: 1855225,
 7032222941422186310: 4376805,
 3898410986347954178: 3358559,
 3145247326368082605: 644382,
 111435857826676758: 194134,
 2584200619530833455: 4949719,
 7180780963176120344: 413714,
 5773588068518854685: 1290435,
 4259658144088588320: 3431187,
 4361844474410872292: 3396305,
 2800019288864325667: 876548,
 9189581406794154018: 3124959,
 7914380015729377316: 4242223,
 698457381071224872: 3163144,
 4982010528525189159: 143239,
 3463847689572057129: 30840,
 6673682719827820585: 3215635,
 8497399280737189929: 4834589,
 8904694660140630058: 1089836,
 2883314632905195565: 72933,
 5643129746290464091: 3323873,
 1778977376621822002: 4781332,
 8332832109865467952: 1462286,
 1315449813499641908: 3886393,
 4916540014084489267: 213196,
 1836925014956310582: 9

In [8]:
for i in range(len(events_df)):
    hashing_value = events_df["session_id"].values[i]
    events_df["session_id"].values[i] = session_id_map[hashing_value]

NameError: name 'session_id_map' is not defined

In [11]:
# encoding event_value
event_value_ct = events_df.event_value.nunique()
unique_event_values = list(events_df.event_value.unique())
event_value_mapping = list(np.arange(0, event_value_ct, 1))
event_value_dict = {}
for u,m in zip(unique_event_values, event_value_mapping):
    event_value_dict[u] = m

In [12]:
event_value_dict

{0.0: 0,
 1.0: 1,
 1.3930000066757202: 13,
 2.0: 17,
 3.4929997920989986: 7,
 4.0: 19,
 5.0: 4,
 6.992999999999999: 24,
 6.993000030517577: 18,
 7.0: 15,
 8.0: 54,
 9.0: 55,
 10.0: 16,
 11.0: 76,
 12.0: 2,
 13.0: 44,
 13.993: 27,
 14.0: 14,
 15.0: 10,
 16.0: 12,
 17.0: 8,
 18.0: 6,
 19.0: 5,
 20.0: 3,
 21.0: 42,
 22.0: 68,
 23.0: 34,
 24.0: 43,
 25.0: 9,
 27.993002: 23,
 29.0: 69,
 30.0: 11,
 nan: 79,
 1.393: 20,
 3.0: 26,
 3.4930000000000003: 21,
 6.0: 56,
 13.993000030517578: 22,
 27.99300193786621: 31,
 32.0: 71,
 35.0: 59,
 36.0: 25,
 40.0: 37,
 48.0: 29,
 50.0: 33,
 60.0: 82,
 69.99299599999999: 35,
 69.99299621582031: 32,
 965.0: 57,
 985.0: 58,
 99981.0: 41,
 999969.0: 40,
 999981.0: 52,
 999993.0: 53,
 9999969.0: 75,
 99999984.0: 48,
 429497290.0: 47,
 999999954.0: 49,
 999999969.0: 80,
 999999974.0: 50,
 999999989.0: 67,
 4293967267.0: 73,
 4293967279.0: 74,
 4294967046.0: 70,
 4294967146.0: 83,
 4294967225.0: 77,
 4294967231.0: 46,
 4294967236.0: 30,
 4294967242.0: 84,
 42949

In [13]:
# encoding event
event_ct = events_df.event.nunique()
unique_event = list(events_df.event.unique())
event_mapping = list(np.arange(0, event_ct, 1))
event_dict = {}
for u,m in zip(unique_event, event_mapping):
    event_dict[u] = m

In [14]:
event_dict

{'.a4575955121471488': 84,
 '.a4640834587066368': 77,
 '.a4767693735067648': 97,
 '.a4809797501583360': 59,
 '.a4905497451167744': 70,
 '.a4920882459443200': 89,
 '.a4936179179192320': 63,
 '.a4955822838251520': 73,
 '.a5008055943430144': 32,
 '.a5027911885258752': 7,
 '.a5061295285075968': 24,
 '.a5087083622694912': 85,
 '.a5400102822346752': 9,
 '.a5516611293544448': 8,
 '.a5523274289512448': 30,
 '.a5678563962912768': 79,
 '.a5686782234722304': 29,
 '.a5924766976704512': 78,
 '.a5933275901722624': 10,
 '.a6017451431428096': 96,
 '.a6030168535203840': 102,
 '.a6031326957797376': 64,
 '.a6035220340277248': 60,
 '.a6266265261441024': 104,
 '.a6278994001788928': 71,
 '.a6544973222248448': 88,
 '.a6666568788738048': 92,
 '.c4673994986356736 Enter': 69,
 '.c4673994986356736 Exit': 67,
 '.m4545368715427840': 76,
 '.m4545368715427840 Accept': 105,
 '.m4592402159501312': 87,
 '.m4609541589565440': 31,
 '.m4609541589565440 Accept': 58,
 '.m4609541589565440 Held Back': 56,
 '.m4728338570018816

In [15]:
for i in range(len(events_df)):
    hashing_value = events_df["event_value"].values[i]
    events_df["event_value"].values[i] = event_value_dict[hashing_value]
    hashing_value_2 = events_df["event"].values[i]
    events_df["event"].values[i] = event_dict[hashing_value_2]

KeyError: nan

In [28]:
# for i in range(len(events_df)):
#     hashing_value = events_df["event"].values[i]
#     events_df["event"].values[i] = event_dict[hashing_value]

KeyError: 0

In [29]:
# save the user_id_map for events and attributes
np.save('session_id_map.npy', session_id_map) 

MemoryError: 

In [ ]:
# save the event_value_map for events and attributes
np.save('event_value_map.npy', event_value_dict) 

In [ ]:
np.save('event_map.npy', event_dict) 

In [16]:
events_df.to_feather("/tmp/events")

In [4]:
events_df = pd.read_feather("/tmp/events")

In [5]:
events_df.head()

,session_id,event,event_timestamp,event_value,user_id_hash
0,0,45,428393.165870,0.0,0
1,0,45,428393.190249,0.0,0
2,1,.m5100869650219008,428090.113992,0.0,0
3,2,4,428809.191980,0.0,0
4,2,6,428809.192532,0.0,0


In [6]:
events_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 5 columns):
session_id         int64
event              object
event_timestamp    float64
event_value        float64
user_id_hash       int64
dtypes: float64(2), int64(2), object(1)
memory usage: 9.7 GB


In [ ]:
# events_df.to_csv("/home/zxr_nju/data/new_events.csv", index=False)

In [7]:
events_df.to_csv("/home/zxr_nju/data/new_events.zip", index=False, compression="zip")

MemoryError: 

In [37]:
# events_df.to_feather("/tmp/events_df")

In [23]:
types = {
    "app_id":np.uint64, "session_id":np.uint64, "event":"category", "event_timestamp": np.uint64,
    "event_value": np.float64, "user_id_hash": np.uint32
}

In [17]:
events.to_csv("/home/zxr_nju/data/new_events.csv", index=False)

OSError: [Errno 28] No space left on device

In [24]:
events_short = pd.read_csv("/home/zxr_nju/data/new_events.csv", usecols=types.keys(), dtype=types)

MemoryError: 

In [ ]:
print(events_short.info(memory_usage="deep"))

In [14]:
events.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 6 columns):
app_id             int64
session_id         int64
event              object
event_timestamp    int64
event_value        float64
user_id_hash       object
dtypes: float64(1), int64(3), object(2)
memory usage: 13.8 GB


In [15]:
events.to_feather("/tmp/events")

In [16]:
events.describe()

,app_id,session_id,event_timestamp,event_value
count,1.119466e+08,1.119466e+08,1.119466e+08,1.119465e+08
mean,4.724683e+15,4.615011e+18,1.541643e+12,1.476199e+04
std,1.056559e+07,2.662939e+18,1.639665e+09,7.948289e+06
min,4.724683e+15,2.335879e+12,1.538377e+12,0.000000e+00
25%,4.724683e+15,2.310347e+18,1.540306e+12,0.000000e+00
50%,4.724683e+15,4.617399e+18,1.541696e+12,0.000000e+00
75%,4.724683e+15,6.921542e+18,1.542983e+12,0.000000e+00
max,4.724683e+15,9.223371e+18,1.544832e+12,4.294967e+09


In [ ]:
# counter = 0
# for i in range(len(events)):
#     if events['user_id_hash'].values[i] not in user_map:
#         user_map[events['user_id_hash'].values[i]] = counter
#         counter += 1
#     events['user_map'] = user_map[events['user_id_hash'].values[i]]